<a href="https://colab.research.google.com/github/timonpalm/TFCourse/blob/main/08_introduction_2_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

# Check for GPU

In [123]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-09db3c1d-4033-85b8-e97d-da0786088af2)


## Get Helper functions

In [124]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-01 18:40:16--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.3’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-01-01 18:40:16 (93.0 MB/s) - ‘helper_functions.py.3’ saved [10246/10246]



## Get a text dataset

Teh dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of Tweets labelled as disaster or not disaster)

See https://www.kaggle.com/competitions/nlp-getting-started/overview

In [125]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-01-01 18:40:16--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 2607:f8b0:4023:c0d::cf, 2607:f8b0:4023:c03::cf
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.3’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-01-01 18:40:17 (109 MB/s) - ‘nlp_getting_started.zip.3’ saved [607343/607343]



## Visualizing a text datset

In [126]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [127]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [128]:
train_df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [129]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [130]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [131]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [132]:
train_df.sample(n=5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
7320,10479,wild%20fires,NaN,@EnzasBargains A5 Donated some fruit snacks &a...,1
4285,6088,hellfire,NaN,Hellfire is surrounded by desires so be carefu...,0
6935,9945,trouble,North Carolina,@TJ_Robertson2 no bc we always got in trouble ...,0
2447,3513,derailment,"Chicago, IL 60607",Still and Box alarm for the train derailment a...,1


In [133]:
import random
random_index = random.randint(0, len(train_df)-5)
for _, text, target in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  print(f"Target: {target}", "(real disaster)" if target > 0  else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
when your moms being annoying so you turn your beats pill all the way up to drown her out.

---

Target: 1 (real disaster)
Text:
Wreckage 'Conclusively Confirmed' as From MH370: Malaysia PM: Investigators and the families of those who were... http://t.co/2Jr3Yo55dr

---

Target: 0 (not real disaster)
Text:
waiting for my chocolate lava cakes to get here ??????

---

Target: 1 (real disaster)
Text:
Maryland mansion fire that killed 6 caused by damaged plug under Christmas tree report says - Into the flames... http://t.co/P1WmkXA9d8

---

Target: 0 (not real disaster)
Text:
@MikeParrActor has confirmed on his twitter saying goodbye 2 ross. Am bloody gobsmacked/devastated #emmerdale

---



### Split data into training and validation

In [134]:
from sklearn.model_selection import train_test_split

In [135]:
# Use train_test_split to split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [136]:
# Checl the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [137]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text Vectorization

In [138]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# Use the deafult TextVerctorization
text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int",
                                    output_sequence_length=None, # length of the sequences
                                    pad_to_max_tokens=True)

In [139]:
# Find the average number of tokens (words) in th training tweets
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [140]:
# Setup text vectorization varaibles
max_vocab_length = 10000 # max number of words in vocab
max_length = 15 # max length our sequences will be (how many words of the tweets will the model see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [141]:
# Fit the text vectorizor to the train data
text_vectorizer.adapt(train_sentences)

In [142]:
# Create a sample sentence and tokinize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [143]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVecorized version:")
text_vectorizer([random_sentence])

Original text:
 The last few days of summer are supposed to be the most fun so what's more fun then accidentally burning arm hair while playing w/ a lighter        

Vecorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2,  139,  548,  581,    6,  270,   22, 1463,    5,   21,    2,
         230,  604,   28,  664]])>

In [144]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

In [145]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                              output_dim=128,
                              input_length=max_length)

embedding

In [146]:
# Get a radnom sentence from the training set
random_sentence = random.choice(train_sentences)

print(f"Original text:\n {random_sentence}\
        \n\nEmbedding version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @Ashayo @MsMiggi Hi Ashayo! I believe there will be VODs on YouTube after the presentation but there is nothing like seeing it live :)        

Embedding version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02152348, -0.00405939, -0.03220787, ..., -0.00750818,
          0.03791573, -0.03977559],
        [ 0.0397223 , -0.03980187, -0.01534698, ..., -0.03853256,
         -0.02604811, -0.00376765],
        [-0.04877306, -0.01288308, -0.02701516, ..., -0.02433854,
         -0.03509215,  0.03030818],
        ...,
        [ 0.03595207,  0.01929513, -0.03073003, ..., -0.03944044,
          0.01081842, -0.02846551],
        [-0.0484849 ,  0.01604304,  0.02945629, ..., -0.04887417,
          0.01848719, -0.01704881],
        [ 0.01193755, -0.03824443,  0.0087347 , ..., -0.03254156,
          0.03066644,  0.04401577]]], dtype=float32)>

In [147]:
# Check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.02152348, -0.00405939, -0.03220787,  0.04412237,  0.00499482,
        -0.00419763, -0.01298506, -0.00043903, -0.01043409,  0.00670966,
         0.02124869, -0.00244782, -0.04807205, -0.01679854, -0.00498956,
        -0.00725825,  0.0378915 , -0.02724235,  0.01249657, -0.02902222,
         0.02359351,  0.03371504, -0.0003052 , -0.00243242, -0.02317022,
        -0.01597495, -0.0286441 , -0.0451692 ,  0.00692058,  0.04488662,
        -0.03300609,  0.01209928, -0.04313536,  0.03743769,  0.0246501 ,
        -0.03359666,  0.03839966,  0.0247235 , -0.03618474,  0.00208632,
        -0.0189557 , -0.02476016,  0.01794254, -0.00988678,  0.01683554,
        -0.03333171,  0.02041421,  0.00926188,  0.00786363, -0.03230882,
         0.04157258, -0.03154936,  0.02283538,  0.02232459, -0.03103751,
        -0.02483591, -0.01059651, -0.03582013, -0.02047509,  0.04449115,
        -0.04715976,  0.02359634,  0.03325912,  0.02922839, -0.01497054,
  

## Modelling a text dataset (sunning a series of experiments)

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline)
* Model 1: Feed Forward neural network (dense model)
* Model 2: LSTM (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

### Model 0: Getting a baseline

In [148]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [149]:
# Evaluate the baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [150]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

### Creating a evaluation function

In [151]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def calculate_results(y_true, y_pred):
  res = {
      "accuracy": accuracy_score(y_true, y_pred),
      "precision": precision_score(y_true, y_pred, average="weighted"),
      "recall": recall_score(y_true, y_pred, average="weighted"),
      "f1": f1_score(y_true, y_pred, average="weighted")
  }

  return res

In [152]:
baseline_results = calculate_results(val_labels, baseline_preds)
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Feed Forward

In [153]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dim strings
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs=inputs, outputs=outputs, name="model_1_dense")

In [154]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_7 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_2  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [155]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [156]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 24s 103ms/step - loss: 0.6111 - accuracy: 0.6891 - val_loss: 0.5348 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 7s 32ms/step - loss: 0.4415 - accuracy: 0.8186 - val_loss: 0.4674 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.3464 - accuracy: 0.8591 - val_loss: 0.4539 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2836 - accuracy: 0.8901 - val_loss: 0.4680 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 5s 21ms/step - loss: 0.2371 - accuracy: 0.9129 - val_loss: 0.4765 - val_accuracy: 0.7769


In [157]:
preds = model_1.predict(val_sentences)
preds.shape

24/24 [==============================] - 0s 6ms/step


(762, 1)

In [158]:
preds = tf.squeeze(tf.round(preds))

In [159]:
model_1_results = calculate_results(val_labels, preds)
model_1_results

{'accuracy': 0.7769028871391076,
 'precision': 0.7781874595396938,
 'recall': 0.7769028871391076,
 'f1': 0.7751243211017478}

## Visualizing learned embeddings

In [160]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [161]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_7 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_2  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [163]:
# Get the weight matrix of embedding layer
# (these are the numerical respresentations of each token in our training data)
embed_weights = model_1.get_layer("embedding_3").get_weights()[0]
embed_weights.shape

(10000, 128)

Use tensorflow projector [tool](https://projector.tensorflow.org/) to visualize the embeddings. Find the code [here](https://www.tensorflow.org/text/guide/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk)

In [164]:
# import io

# out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
# out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

# for index, word in enumerate(words_in_vocab):
#   if index == 0:
#     continue  # skip 0, it's padding.
#   vec = embed_weights[index]
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
#   out_m.write(word + "\n")
# out_v.close()
# out_m.close()

In [165]:
# # download files to upload to projector
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks (RNN)

### Model 2: LSTM

In [170]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x) # when you'are stacking LSTM cells together you need the sequences
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name="model_2_lstm")

In [171]:
model_2.summary()

Model: "model_2_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_7 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_4 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                      

In [172]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [175]:
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 13s 37ms/step - loss: 0.2199 - accuracy: 0.9261 - val_loss: 0.6062 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1553 - accuracy: 0.9423 - val_loss: 0.6567 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.1294 - accuracy: 0.9510 - val_loss: 0.8599 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1056 - accuracy: 0.9590 - val_loss: 0.8346 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0855 - accuracy: 0.9670 - val_loss: 1.1000 - val_accuracy: 0.7638


In [176]:
pred_probs = model_2.predict(val_sentences)
pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[3.8065314e-03],
       [6.2643951e-01],
       [9.9997985e-01],
       [9.0946453e-03],
       [1.8025628e-04],
       [9.9589324e-01],
       [9.3412775e-01],
       [9.9999225e-01],
       [9.9998903e-01],
       [3.5058269e-01]], dtype=float32)

In [179]:
pred = tf.squeeze(tf.round(pred_probs))
model_2_results = calculate_results(val_labels, preds)
model_2_results

{'accuracy': 0.7769028871391076,
 'precision': 0.7781874595396938,
 'recall': 0.7769028871391076,
 'f1': 0.7751243211017478}